In [1]:
import torch, torchvision, matplotlib
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch.nn as nn
torch.manual_seed(0)

from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np
np.random.seed(0)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [0]:
# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='MNIST/data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='MNIST/data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


In [0]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU()) #No pooling this layer
        
        self.fc = nn.Linear(7*7*16, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [5]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.0902
Epoch [1/5], Step [200/600], Loss: 0.0905
Epoch [1/5], Step [300/600], Loss: 0.0573
Epoch [1/5], Step [400/600], Loss: 0.0773
Epoch [1/5], Step [500/600], Loss: 0.0699
Epoch [1/5], Step [600/600], Loss: 0.0999
Epoch [2/5], Step [100/600], Loss: 0.0605
Epoch [2/5], Step [200/600], Loss: 0.0750
Epoch [2/5], Step [300/600], Loss: 0.0280
Epoch [2/5], Step [400/600], Loss: 0.0331
Epoch [2/5], Step [500/600], Loss: 0.0218
Epoch [2/5], Step [600/600], Loss: 0.0750
Epoch [3/5], Step [100/600], Loss: 0.0025
Epoch [3/5], Step [200/600], Loss: 0.0041
Epoch [3/5], Step [300/600], Loss: 0.0603
Epoch [3/5], Step [400/600], Loss: 0.0139
Epoch [3/5], Step [500/600], Loss: 0.0415
Epoch [3/5], Step [600/600], Loss: 0.0016
Epoch [4/5], Step [100/600], Loss: 0.0361
Epoch [4/5], Step [200/600], Loss: 0.0070
Epoch [4/5], Step [300/600], Loss: 0.0261
Epoch [4/5], Step [400/600], Loss: 0.0496
Epoch [4/5], Step [500/600], Loss: 0.0109
Epoch [4/5], Step [600/600], Loss:

In [0]:
path = F"/content/gdrive/My Drive/correct_test_dataset.txt"
correct_test = np.loadtxt(path, delimiter=",", dtype=int)

In [0]:
# Dictionary to record which indices are incorrectly classified
# incorrection_dict[true_label] = what model thinks the class is
incorrection_dict = {}
incorrection_dict[0] = []
incorrection_dict[1] = []
incorrection_dict[2] = []
incorrection_dict[3] = []
incorrection_dict[4] = []
incorrection_dict[5] = []
incorrection_dict[5] = []
incorrection_dict[6] = []
incorrection_dict[7] = []
incorrection_dict[8] = []
incorrection_dict[9] = []

In [8]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    for idx, (images, labels) in enumerate(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        #only test ones correctly classified by the original model
        for j in range(len(predicted)):
          if idx*batch_size+j in correct_test:
            if labels[j] == predicted[j]:
              correct +=1
            else:
              incorrection_dict[labels[j].item()].append(predicted[j].item())

    total = len(correct_test)
        
    print('Test Accuracy of the model on the {} test images (100% for original model): {} %'.format(total, 100 * correct / total))


Test Accuracy of the model on the 9896 test images (100% for original model): 99.31285367825384 %


In [9]:
print(correct/total)

0.9931285367825384


In [10]:
incorrection_dict

{0: [],
 1: [2, 3, 3, 3, 3, 2, 2, 8, 0, 0, 2, 0],
 2: [0, 0, 3, 0, 7],
 3: [],
 4: [6],
 5: [3, 3, 3, 0, 3, 3, 3],
 6: [0, 0, 1, 5, 8, 0, 5, 3, 0, 5, 3],
 7: [2, 3, 3, 2],
 8: [2, 3, 3, 2, 2, 9, 3],
 9: [3, 3, 0, 4, 3, 4, 4, 0, 4, 3, 4, 0, 7, 3, 4, 1, 4, 3, 3, 7, 7]}

In [0]:
path = F"/content/gdrive/My Drive/correction_dict.npy" 
correction_dict=np.load(path, allow_pickle=True)

In [12]:
def calc_mutation_score():
  score = 0
  for key, value in incorrection_dict.items():
    count_misclassify = len(value)
    normalizing = len(correction_dict.item().get(key))
    score += count_misclassify/normalizing
  return score
calc_mutation_score()

0.06847152332122901